In [1]:
import cv2
import dlib
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
SCALE_FACTOR = 1 
FEATHER_AMOUNT = 11

FACE_POINTS = list(range(17, 68))
MOUTH_POINTS = list(range(48, 61))
RIGHT_BROW_POINTS = list(range(17, 22))
LEFT_BROW_POINTS = list(range(22, 27))
RIGHT_EYE_POINTS = list(range(36, 42))
LEFT_EYE_POINTS = list(range(42, 48))
NOSE_POINTS = list(range(27, 35))
JAW_POINTS = list(range(0, 17))

# Points used to line up the images.
ALIGN_POINTS = (LEFT_BROW_POINTS + RIGHT_EYE_POINTS + LEFT_EYE_POINTS +
                               RIGHT_BROW_POINTS + NOSE_POINTS + MOUTH_POINTS)

# Points from the second image to overlay on the first. The convex hull of each
# element will be overlaid.
OVERLAY_POINTS = [list(range(0, 68))]

# Amount of blur to use during colour correction, as a fraction of the
# pupillary distance.
COLOUR_CORRECT_BLUR_FRAC = 1.4

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)
dst_dirname = 'dst'
src_dirname = 'src'
output_dirname = 'output'

dst_files = os.listdir(dst_dirname)
src_files = os.listdir(src_dirname)

In [3]:
def get_landmarks(im):
    rects = detector(im, 1)
    
    return np.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])

def annotate_landmarks(im, landmarks):
    im = im.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        cv2.putText(im, str(idx), pos,
                    fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                    fontScale=0.4,
                    color=(0, 0, 255))
        cv2.circle(im, pos, 3, color=(0, 255, 255))
    return im

def draw_convex_hull(im, points, color):
    points = cv2.convexHull(points)
    cv2.fillConvexPoly(im, points, color=color)

def get_face_mask(im, landmarks):
    im = np.zeros(im.shape[:2], dtype=np.float64)

    for group in OVERLAY_POINTS:
        draw_convex_hull(im,
                         landmarks[group],
                         color=1)

    im = np.array([im, im, im]).transpose((1, 2, 0))

    im = (cv2.GaussianBlur(im, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0) > 0) * 1.0
    im = cv2.GaussianBlur(im, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0)

    return im
    
def transformation_from_points(points1, points2):
    """
    Return an affine transformation [s * R | T] such that:
        sum ||s*R*p1,i + T - p2,i||^2
    is minimized.
    """
    # Solve the procrustes problem by subtracting centroids, scaling by the
    # standard deviation, and then using the SVD to calculate the rotation. See
    # the following for more details:
    #   https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem

    points1 = points1.astype(np.float64)
    points2 = points2.astype(np.float64)

    c1 = np.mean(points1, axis=0)
    c2 = np.mean(points2, axis=0)
    points1 -= c1
    points2 -= c2

    s1 = np.std(points1)
    s2 = np.std(points2)
    points1 /= s1
    points2 /= s2

    U, S, Vt = np.linalg.svd(points1.T * points2)

    # The R we seek is in fact the transpose of the one given by U * Vt. This
    # is because the above formulation assumes the matrix goes on the right
    # (with row vectors) where as our solution requires the matrix to be on the
    # left (with column vectors).
    R = (U * Vt).T

    return np.vstack([np.hstack(((s2 / s1) * R,
                                       c2.T - (s2 / s1) * R * c1.T)),
                         np.matrix([0., 0., 1.])])

def read_im_and_landmarks(fname):
    im = cv2.imread(fname, cv2.IMREAD_COLOR)
    im = cv2.resize(im, (im.shape[1] * SCALE_FACTOR,
                         im.shape[0] * SCALE_FACTOR))
    s = get_landmarks(im)

    return im, s

def warp_im(im, M, dshape):
    output_im = np.zeros(dshape, dtype=im.dtype)
    cv2.warpAffine(im,
                   M[:2],
                   (dshape[1], dshape[0]),
                   dst=output_im,
                   borderMode=cv2.BORDER_TRANSPARENT,
                   flags=cv2.WARP_INVERSE_MAP)
    return output_im

def correct_colours(im1, im2, landmarks1):
    blur_amount = COLOUR_CORRECT_BLUR_FRAC * np.linalg.norm(
                              np.mean(landmarks1[LEFT_EYE_POINTS], axis=0) -
                              np.mean(landmarks1[RIGHT_EYE_POINTS], axis=0))
    blur_amount = int(blur_amount)
    if blur_amount % 2 == 0:
        blur_amount += 1
    im1_blur = cv2.GaussianBlur(im1, (blur_amount, blur_amount), 0)
    im2_blur = cv2.GaussianBlur(im2, (blur_amount, blur_amount), 0)

    # Avoid divide-by-zero errors.
    im2_blur += (128 * (im2_blur <= 1.0)).astype(im2_blur.dtype)

    return (im2.astype(np.float64) * im1_blur.astype(np.float64) /
                                                im2_blur.astype(np.float64))

def face_swap(dst, src):
    im1, landmarks1 = read_im_and_landmarks(dst)
    im2, landmarks2 = read_im_and_landmarks(src)
    
    M = transformation_from_points(landmarks1[ALIGN_POINTS],
                               landmarks2[ALIGN_POINTS])
    
    mask = get_face_mask(im2, landmarks2)
    warped_mask = warp_im(mask, M, im1.shape)
    combined_mask = get_face_mask(im1, landmarks1)
    

    warped_im2 = warp_im(im2, M, im1.shape)
    warped_corrected_im2 = correct_colours(im1, warped_im2, landmarks1)
    output_im = im1 * (1.0 - combined_mask) + warped_corrected_im2 * combined_mask
    
    return output_im

    
def mse(imageA, imageB):
	# the 'Mean Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	
	# return the MSE, the lower the error, the more "similar"
	# the two images are
	return err

def find_best_face(dst):   
    E = []
    dst_path = os.path.join(dst_dirname, dst) 
    im1, landmarks1 = read_im_and_landmarks(dst_path)
    dst_mask = mask = get_face_mask(im1, landmarks1)
    
    for index, src in enumerate(src_files):        
        im2 = src_images[index]
        landmarks2 = landmarks[index]        
        M = transformation_from_points(landmarks1[ALIGN_POINTS],
                               landmarks2[ALIGN_POINTS])
        
        mask = masks[index]
        warped_mask = warp_im(mask, M, im1.shape)
        combined_mask = np.max([get_face_mask(im1, landmarks1), warped_mask], axis=0)
        e = mse(combined_mask, dst_mask)
        E.append(e)
    
    best_src = src_files[np.argmin(E)]
    best_src_path = src_path = os.path.join(src_dirname, best_src)
    best_image = face_swap(dst_path, best_src_path)
    file_name = dst.split('.')[0] + best_src.split('.')[0] + '.' + dst.split('.')[1]  
    f = os.path.join(output_dirname, file_name) 
    cv2.imwrite(f, best_image)
    return np.argmin(E)

def face_swaps(dst_files):
    for dst in dst_files:
        print(dst)
        try:
            find_best_face(dst)
        except:
            pass

In [4]:
def clean_files(dirname, files):
    print('cleaning...')
    for file in files:
        f = os.path.join(dirname, file)
        try:
            read_im_and_landmarks(f)
        except:
            print(f)
            os.remove(f)              
            
def refresh_dir():
    print('refreshing')
    dst_files = os.listdir(dst_dirname)
    src_files = os.listdir(src_dirname)

In [5]:
if 1+1==2:
    clean_files(dst_dirname, dst_files)
    refresh_dir()
    print('Done')

cleaning...
dst\jasmine_jae (103).jpg
dst\jasmine_jae (104).jpg
dst\jasmine_jae (105).jpg
dst\jasmine_jae (106).jpg
dst\jasmine_jae (107).jpg
dst\jasmine_jae (108).jpg
dst\jasmine_jae (109).jpg
dst\jasmine_jae (110).jpg
dst\jasmine_jae (111).jpg
dst\jasmine_jae (112).jpg
dst\jasmine_jae (113).jpg
dst\jasmine_jae (114).jpg
dst\jasmine_jae (115).jpg
dst\jasmine_jae (116).jpg
dst\jasmine_jae (122).jpg
dst\jasmine_jae (132).jpg
dst\jasmine_jae (133).jpg
dst\jasmine_jae (134).jpg
dst\jasmine_jae (136).jpg
dst\jasmine_jae (138).jpg
dst\jasmine_jae (143).jpg
dst\jasmine_jae (145).jpg
dst\jasmine_jae (146).jpg
dst\jasmine_jae (151).jpg
dst\jasmine_jae (160).jpg
dst\jasmine_jae (168).jpg
dst\jasmine_jae (171).jpg
dst\jasmine_jae (183).jpg
dst\jasmine_jae (184).jpg
dst\jasmine_jae (185).jpg
dst\jasmine_jae (187).jpg
dst\jasmine_jae (188).jpg
dst\jasmine_jae (189).jpg
dst\jasmine_jae (190).jpg
dst\jasmine_jae (191).jpg
dst\jasmine_jae (196).jpg
dst\jasmine_jae (205).jpg
dst\jasmine_jae (25).jpg
d

In [ ]:
if 1+1 == 2:
    print("Loading")
    src_images = []
    landmarks = []
    masks = []
    for src in src_files:
        src_path = os.path.join(src_dirname, src)
        im2, landmarks2 = read_im_and_landmarks(src_path)
        mask = get_face_mask(im2, landmarks2)
        src_images.append(im2)
        landmarks.append(landmarks2)
        masks.append(mask)

Loading


In [ ]:
if 1+1 == 2:
    face_swaps(dst_files)

jasmine_jae (1).jpg
jasmine_jae (10).jpg
jasmine_jae (100).jpg
jasmine_jae (101).jpg
jasmine_jae (102).jpg
jasmine_jae (103).jpg
jasmine_jae (104).jpg
jasmine_jae (105).jpg
jasmine_jae (106).jpg
jasmine_jae (107).jpg
jasmine_jae (108).jpg
jasmine_jae (109).jpg
jasmine_jae (11).jpg
jasmine_jae (110).jpg
jasmine_jae (111).jpg
jasmine_jae (112).jpg
jasmine_jae (113).jpg
jasmine_jae (114).jpg
jasmine_jae (115).jpg
jasmine_jae (116).jpg
jasmine_jae (117).jpg
jasmine_jae (118).jpg
jasmine_jae (119).jpg
jasmine_jae (12).jpg
jasmine_jae (120).jpg
jasmine_jae (121).jpg
jasmine_jae (122).jpg
jasmine_jae (123).jpg
jasmine_jae (124).jpg
jasmine_jae (125).jpg
jasmine_jae (126).jpg
jasmine_jae (127).jpg
jasmine_jae (128).jpg
jasmine_jae (129).jpg
jasmine_jae (13).jpg
jasmine_jae (130).jpg
jasmine_jae (131).jpg
jasmine_jae (132).jpg
jasmine_jae (133).jpg
jasmine_jae (134).jpg
jasmine_jae (135).jpg
jasmine_jae (136).jpg
jasmine_jae (137).jpg
jasmine_jae (138).jpg
jasmine_jae (139).jpg
jasmine_jae (14)

In [ ]:
if 1 + 1 == 3:
    clean_dirs()
    refresh_dir()
    print('Done')
    for src in src_files:
        f = os.path.join(src_dirname, src)
        img = cv2.imread(f, cv2.IMREAD_COLOR)
        rimg=cv2.flip(img,1)
        src = src.replace('.', '-r.')
        f = os.path.join(rsrc_dirname, src)
        print(f)
        cv2.imwrite(f, rimg)
    print('Done')

In [ ]:
if 1+1 == 3:
    dst = os.path.join(dst_dirname, dst_files[0])
    src = os.path.join(src_dirname, "sr (53)-r.jpg")
    o = face_swap(dst, src)
    cv2.imwrite('output.jpg', o)